Comparing the actual label with the VADER prediction, we get an accuracy score of 0.768

# Imports

In [43]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

In [44]:
# Imports
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

### Stopwords
Stopwords are words that are very common and add little meaning
examples: a, of, the

In [45]:
stopwords = nltk.corpus.stopwords.words('english')

### PorterStemmer
Words that have the same stem, typically have the same meaning
PorterStemmer cuts off the affixes so you just use the stem -> reduces word count (features)

In [46]:
ps = nltk.PorterStemmer()

# Read in Data

In [47]:
data_news = pd.read_csv("Financial_News_Data_NLP.csv", encoding = "ISO-8859-1")
data_news.columns = ["sentiment", "headlines"]
data_news.head()

,sentiment,headlines
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


In [48]:
len(data_news)

4845

In [49]:
data_twitter_og = pd.read_csv("../Twitter_Data_for_NLP.csv", encoding = "ISO-8859-1")
data_twitter_og.columns = ["tweet", "sentiment"]
data_twitter_og

,tweet,sentiment
0,when modi promised âminimum government maxim...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


### Eliminate Empty tweets! Probably pictures or memes 

In [50]:
data_twitter_og["tweet"] = data_twitter_og["tweet"].astype("string")
data_twitter_cleaned = data_twitter_og.loc[pd.notna(data_twitter_og["tweet"]),:].copy()
data_twitter = data_twitter_cleaned.sample(n=10000)

In [51]:
len(data_twitter)

10000

In [52]:
data_twitter.reset_index(inplace=True)

In [53]:
data_twitter.drop(columns="index", inplace=True)

In [54]:
data_twitter

,tweet,sentiment
0,ohh wow undeclared emergency still rahul gandh...,1.0
1,jiske khoon mein milawat hokya woh humare desh...,-1.0
2,have same experience with modi hatter,0.0
3,when would you change the name your channel modi,0.0
4,which wouldve been seen replay and thus kept p...,1.0
...,...,...
9995,you speak amethi then agree rahul gandhi has d...,1.0
9996,when the entire arunachal stands with their ch...,0.0
9997,religious solution for lok sabha elections yag...,0.0
9998,talking about india under modi hole,0.0


# Prepare Data

## Feature Creation
1. Punctuation percentage
2. Text Length
3. Captialization percentage

In [55]:
# Function to count punctuation
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    if (len(text) - text.count(" ")) ==0:   # Need to avoid dividing by 0
        return 0
    return round(count/(len(text) - text.count(" ")), 3)*100

# Apply function to make new column
data_news['punct%'] = data_news['headlines'].apply(lambda x: count_punct(x))
data_twitter['punct%'] = data_twitter['tweet'].apply(lambda x: count_punct(x))

In [56]:
len(data_twitter["punct%"])

10000

In [57]:
# Determine length of headline and make it a column
data_news['text_len'] = data_news['headlines'].apply(lambda x: len(x) - x.count(" "))

data_twitter['text_len'] = data_twitter['tweet'].apply(lambda x: len(x) - x.count(" "))

In [58]:
# Function to determine capitalization percentage
def capital_percent(text):
    count = sum([1 for char in text if char.isupper()])
    if (len(text) - text.count(" ")) == 0:      # Avoid dividing by 0
        return 0
    return round(count/(len(text) - text.count(" ")), 3)*100

# Apply function to make new column
data_news['capital%'] = data_news['headlines'].apply(lambda x: capital_percent(x))

data_twitter['capital%'] = data_twitter['tweet'].apply(lambda x: capital_percent(x))

## Clean Data

In [59]:
# Function to clean up data
# Eliminate punctuation
# Make everything lowercase
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]  # Use portstemmer
    return text

## Vectorization

### TF-IDF
Inverse Document Frequency Weighting 
* Creates a document-term matrix where the cells contain a weighting of how important that word is to the text
* How much does a word differentiate a text message from othes? Pulls out important but seldom used words

In [16]:
# Instantiate the object and state our parameters. Pass in the function we created to clean the text (clean_text)
tfidf_vect_news = TfidfVectorizer(analyzer=clean_text)

# Fit and transform model 
X_tfidf_news = tfidf_vect_news.fit_transform(data_news['headlines'])

# Create df to see vectorization and concatenated created features
X_tfidf_feat_news = pd.concat([data_news['text_len'], data_news['punct%'], data_news['capital%'], pd.DataFrame(X_tfidf_news.toarray())], axis=1)
X_tfidf_feat_news.head(5)

,text_len,punct%,capital%,0,1,2,3,4,5,6,...,8901,8902,8903,8904,8905,8906,8907,8908,8909,8910
0,160,1.9,0.6,0.039911,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,193,1.6,2.1,0.036051,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,174,0.6,0.6,0.039099,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,163,6.7,1.2,0.038290,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,154,1.3,21.4,0.033612,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
len(X_tfidf_feat_news)

4845

In [60]:
# Instantiate the object and state our parameters. Pass in the function we created to clean the text (clean_text)
tfidf_vect_twitter = TfidfVectorizer(analyzer=clean_text) 

# Fit and transform model 
X_tfidf_twitter = tfidf_vect_twitter.fit_transform(data_twitter['tweet'])



In [61]:
# Create df to see vectorization and concatenated created features
X_tfidf_feat_twitter = pd.concat([data_twitter['text_len'], data_twitter['punct%'], data_twitter['capital%'], pd.DataFrame(X_tfidf_twitter.toarray())], axis=1)
X_tfidf_feat_twitter.head(5)

,text_len,punct%,capital%,0,1,2,3,4,5,6,...,16146,16147,16148,16149,16150,16151,16152,16153,16154,16155
0,201,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,183,0.0,0.0,0.045123,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,32,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40,0.0,0.0,0.153841,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,63,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Count Vectorizer
* Creates a document term matrix where the entry of each cell will be a count of the number of times that word occurred in that document

In [26]:
# Instantiate the object and state our parameters. Pass in the function we created to clean the text (clean_text)
count_vect_news = CountVectorizer(analyzer=clean_text)

# Fit and Transform model
X_count_news = count_vect.fit_transform(data_news['headlines'])

# Create df to see vectorization and concatenated created features
X_count_feat = pd.concat([data_news['text_len'], data_news['punct%'], data_news['capital%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

NameError: name 'count_vect' is not defined

In [ ]:
# Instantiate the object and state our parameters. Pass in the function we created to clean the text (clean_text)
count_vect_twitter = CountVectorizer(analyzer=clean_text)

# Fit and Transform model
X_count_twitter = count_vect_twitter.fit_transform(data_twitter['tweet'])

# Create df to see vectorization and concatenated created features
X_count_feat_twitter = pd.concat([data_twitter['text_len'], data_twitter['punct%'], data_twitter['capital%'], pd.DataFrame(X_count_twitter.toarray())], axis=1)

X_count_feat_twitter.head()

# Parameter Settings

In [ ]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_news_feat, data_news['sentiment'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/Users/jennifershulyak/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/jennifershulyak/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/jennifershulyak/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/jennifershulyak/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,39.483314,1.167703,0.522142,0.087084,None,300,"{'max_depth': None, 'n_estimators': 300}",0.694530,0.688338,0.635707,0.652219,0.663571,0.666873,0.022012,1
8,51.573586,2.324569,1.263336,0.273069,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.696594,0.698658,0.629515,0.639835,0.663571,0.665635,0.028368,2
10,34.212505,2.768572,0.906632,0.386668,None,150,"{'max_depth': None, 'n_estimators': 150}",0.692466,0.691434,0.635707,0.640867,0.665635,0.665222,0.024058,3
5,42.425284,1.016262,1.271919,0.364790,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.713106,0.683179,0.623323,0.631579,0.653251,0.660888,0.033323,4
7,26.763226,1.563173,1.016106,0.264996,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.702786,0.682147,0.630547,0.626419,0.659443,0.660268,0.029381,5


### For Twitter

In [ ]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_twitter_array, data_twitter['sentiment'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

KeyboardInterrupt: 

# Random Forest on Holdout Test Set

## Twitter

In [63]:
# X_tfidf_feat_twitter
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf_feat_twitter, data_twitter['sentiment'], test_size=0.2)

In [65]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

/Users/jennifershulyak/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [70]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred)

/Users/jennifershulyak/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [73]:
round((y_pred==y_test).sum() / len(y_pred),3)

0.576

In [74]:
for x in range(3):
    print(precision[x])

0.8518518518518519
0.7415730337078652
0.5222513089005235


In [68]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

TypeError: type numpy.ndarray doesn't define __round__ method